In [1]:
# import sys
# from  ift6758.data.data_acquisition import Season
# season2018 = Season(2018,"../ift6758/data")
# df_season2018_clean = season2018.clean_data()
# df[["gamePk","liveData.plays.allPlays"]]

In [92]:
from  ift6758.data.data_acquisition import Season

from ipywidgets import widgets, interact, RadioButtons, IntSlider, Output, Layout, Dropdown
from ipywidgets.embed import embed_minimal_html

from scipy.ndimage import gaussian_filter

import plotly.graph_objects as go

import json
import pandas as pd
import numpy as np
from tabulate import tabulate
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns

import requests
import json

In [93]:
def time_played(row):
    """
    return time in mins
    """
    # print(type(row['about.periodTime']))
    if row['result.event'] == 'GOAL' and row['about.period']>3:
        # return row['about.periodTime'] + row['about.periodTime']
        # .date.min()
        min_sec1 = row['about.periodTime'].split(':')
        return float(min_sec1[0]) + float(min_sec1[1])/60
    else:
        # return row['about.periodTime']
        # .date.min()
        min_sec1 = row['about.periodTime'].split(':')
        min_sec2 = row['about.periodTimeRemaining'].split(':')
        return float(min_sec1[0]) + float(min_sec2[0]) + (float(min_sec1[1]) +  float(min_sec2[1]))/60


In [94]:
# select_columns = ["result.event","gamePk","team.name","players","about.period","about.periodTime","about.periodType","about.periodTimeRemaining","coordinates.x","coordinates.y"]

def get_team_time(df_clean):
    sel_columns = ['gamePk','result.event','team.name','about.period','about.periodTime','about.periodTimeRemaining']
    df1 = df_clean[sel_columns].drop_duplicates() 
    df2 = df1.sort_values('about.periodTime', ascending=False).drop_duplicates(['gamePk','about.period'])
    df2['periodTimePlayed'] = df2.apply (lambda row: time_played(row), axis=1)
    df3 = df2.groupby('gamePk')['periodTimePlayed'].sum()
    df4 = df1[['gamePk','team.name']].drop_duplicates()
    df4['gameTime'] = df4.apply(lambda x: df3.get(key = x['gamePk']),axis=1)
    df5 = df4.groupby(['team.name'])['gameTime'].sum()
    total_time = df5.sum()
    return df5.to_dict(),total_time


    

# time_game_dict = {}
# for year in [2016,2017,2018,2019,2020]:
#     season = Season(year,"../ift6758/data")
#     df_clean = season.clean_data()
#     time_game_dict[year] = {}
#     time_game_dict[year]['team'],time_game_dict[year]['total_time'] = get_team_time(df_clean)
#     time_game_dict[year]['teams_played'] = len(time_game_dict[year]['team'])





In [95]:
def sum_bin(out,bin_width):
    for j in range(0,out.shape[0],bin_width):
        for i in range(0,out.shape[1],bin_width):
            bin_sum=0
            for j1 in range(bin_width):
                for i1 in range(bin_width):
                    if j+j1<out.shape[0] and i+i1<out.shape[1]:
                        bin_sum += out[j+j1][i+i1] 
            for j1 in range(bin_width):
                for i1 in range(bin_width):
                    if j+j1<out.shape[0] and i+i1<out.shape[1]:
                        out[j+j1][i+i1] = bin_sum



In [102]:
import numpy as np
def advanced_visualization(year=2018,team_name="Montréal Canadiens",bin_width=1):
# year=2018
# team_name="Montréal Canadiens"
    def bin_coordinates(row):
        """
        To explain the procedure of inverting the co-ordinates.
        """
        if row['coordinates.x'] <0:
            x_bin = -row['coordinates.x']
            y_bin= 42.5-row['coordinates.y']
        else:
            x_bin = row['coordinates.x']
            y_bin= 42.5+row['coordinates.y']
        
        
        return  np.floor(x_bin), np.floor(y_bin)
    def copyToOutput(row,out):
        """
        Setting out array
        """
        out[int(row['y_bin']),int(row['x_bin'])]=row['excess_shots']
        return "copiedToOutput"
    def sum_bin(out,bin_width):
        for j in range(0,out.shape[0],bin_width):
            for i in range(0,out.shape[1],bin_width):
                bin_sum=0
                for j1 in range(bin_width):
                    for i1 in range(bin_width):
                        if j+j1<out.shape[0] and i+i1<out.shape[1]:
                            bin_sum += out[j+j1][i+i1] 
                for j1 in range(bin_width):
                    for i1 in range(bin_width):
                        if j+j1<out.shape[0] and i+i1<out.shape[1]:
                            out[j+j1][i+i1] = bin_sum                

    season = Season(year,"../ift6758/data")
    df_clean = season.clean_data()
    rink_size = (85,100)
    out = np.zeros(rink_size)

    num_teams = df_clean['team.name'].unique().shape[0]
    print(f"num_teams {num_teams}")
        
    ## Reference: https://stackoverflow.com/a/52363890
    coordinates_df = df_clean.apply(lambda row: bin_coordinates(row),axis='columns', result_type='expand').rename({0: 'x_bin', 1: 'y_bin'}, axis=1)
    df_clean = pd.concat([df_clean, coordinates_df], axis='columns')
    num_games_by_team = df_clean.groupby('team.name')['gamePk'].count().to_dict()
    df_group1 = df_clean.groupby(['team.name','x_bin','y_bin'])['result.event'].count().reset_index()
    # df_group1['shots_average'] = df_group1['result.event']
    df_group1['shots_average'] = df_group1.apply(lambda r: r['result.event'] /num_games_by_team[r['team.name']],axis=1)
    df2 = df_group1.drop('result.event',axis=1).drop_duplicates()
    df_group2 = df2.groupby(['x_bin','y_bin'])['shots_average'].sum()/num_teams
    avg_shots_coordinate_dict = df_group2.to_dict()
    df2['excess_shots'] = df2.apply(lambda row: row['shots_average'] - avg_shots_coordinate_dict[(row['x_bin'],row['y_bin'])],axis=1)
    df3 = df2[df2['team.name']==team_name]

    df3.apply(lambda row: copyToOutput(row,out),axis=1)
    sum_bin(out,bin_width)
    print(df3[(df3['x_bin']==75.0)&(df3['y_bin']==36.0)])
    return out



In [101]:
out1 = advanced_visualization(year=2016,team_name="New Jersey Devils",bin_width=1)
f = gaussian_filter(out1, sigma=2)

File already Exists, loading from ../ift6758/data/PICKLE//2016_clean.pkl
num_teams 34
               team.name  x_bin  y_bin  shots_average  excess_shots
33777  New Jersey Devils   75.0   36.0       0.000406     -0.000051


In [103]:
IMG_PATH = "../figures/half_nhl_rink_h.png"

In [104]:
url = 'https://statsapi.web.nhl.com/api/v1/teams'

r = requests.get(url, headers={'accept': 'application/json'})
json_teams = r.json()

print(json_teams["teams"])

[{'id': 1, 'name': 'New Jersey Devils', 'link': '/api/v1/teams/1', 'venue': {'name': 'Prudential Center', 'link': '/api/v1/venues/null', 'city': 'Newark', 'timeZone': {'id': 'America/New_York', 'offset': -4, 'tz': 'EDT'}}, 'abbreviation': 'NJD', 'teamName': 'Devils', 'locationName': 'New Jersey', 'firstYearOfPlay': '1982', 'division': {'id': 18, 'name': 'Metropolitan', 'nameShort': 'Metro', 'link': '/api/v1/divisions/18', 'abbreviation': 'M'}, 'conference': {'id': 6, 'name': 'Eastern', 'link': '/api/v1/conferences/6'}, 'franchise': {'franchiseId': 23, 'teamName': 'Devils', 'link': '/api/v1/franchises/23'}, 'shortName': 'New Jersey', 'officialSiteUrl': 'http://www.newjerseydevils.com/', 'franchiseId': 23, 'active': True}, {'id': 2, 'name': 'New York Islanders', 'link': '/api/v1/teams/2', 'venue': {'name': 'UBS Arena', 'link': '/api/v1/venues/null', 'city': 'Elmont', 'timeZone': {'id': 'America/New_York', 'offset': -4, 'tz': 'EDT'}}, 'abbreviation': 'NYI', 'teamName': 'Islanders', 'locat

In [105]:
team_list = list()
for team in json_teams["teams"]:
    team_list.append(team["name"])
print(team_list)

['New Jersey Devils', 'New York Islanders', 'New York Rangers', 'Philadelphia Flyers', 'Pittsburgh Penguins', 'Boston Bruins', 'Buffalo Sabres', 'Montréal Canadiens', 'Ottawa Senators', 'Toronto Maple Leafs', 'Carolina Hurricanes', 'Florida Panthers', 'Tampa Bay Lightning', 'Washington Capitals', 'Chicago Blackhawks', 'Detroit Red Wings', 'Nashville Predators', 'St. Louis Blues', 'Calgary Flames', 'Colorado Avalanche', 'Edmonton Oilers', 'Vancouver Canucks', 'Anaheim Ducks', 'Dallas Stars', 'Los Angeles Kings', 'San Jose Sharks', 'Columbus Blue Jackets', 'Minnesota Wild', 'Winnipeg Jets', 'Arizona Coyotes', 'Vegas Golden Knights', 'Seattle Kraken']


In [106]:
#CHOOSE GAME ID
season_selector = IntSlider(
    value=2016,
    min=2016,
    max=2020,
    step=1,
    description='Season: ',
    continuous_update=False, #only reassign value when the slider stops moving
    layout=Layout(width='50%')
)

#CHOOSE GAME TYPE
team_selector = Dropdown(
    options=team_list,
    value='New Jersey Devils',
    description='Number:',
    disabled=False,
    continuous_update=False,
    layout=Layout(width='50%')
)

container = widgets.HBox(children=[season_selector, team_selector])

##

from random import randint
import plotly.express as px
import plotly.graph_objects as go
from skimage import io

#Z = np.random.uniform(-1,1,(85,100))
#trace1 = go.Histogram(x=Z, opacity=0.75, name='Departure Delays')

layout = go.Layout(title='Loading')

img = io.imread(IMG_PATH)
# figure = px.imshow(img,
#                    extent=[],
#                    #x=np.arange(-42.5,43.5),
#                    #y=np.arange(-100,101),
#                    aspect='auto',
#                   )

#     scaleanchor = "x",
#     scaleratio = 1
# )

figure = go.Figure(layout_xaxis_range=[0,100],
                  layout_yaxis_range=[0,85])

figure.update_xaxes(showgrid=False)
figure.update_yaxes(showgrid=False)

figure.update_layout(
    title="Choose a Team and/or a Season",
    xaxis_title="Distance from the Rink's Center Line (feet)",
    yaxis_title="Distance from the Rink's Bottom Edge (feet)",
    legend_title="Legend Title",
    font=dict(
        size=10,
        family='Arial, sans-serif'
    ),
    width=550,
    height=467,
)
#img size: 550 x 467

colorscale = [[0, 'darkblue'],[0.5,"rgba(255, 255, 255, 1)"], [1, 'maroon']]

figure.add_trace (
    go.Contour(
        z=f,
        # cmapk= 'magma',
        colorscale=colorscale,
        line_smoothing=1,
        opacity=0.6,
        connectgaps=True,
        #colorbar setup
        colorbar=dict(
            title='Excess Shots per Hour', # title here
            titleside='right',
            titlefont=dict(
                size=10,
                family='Arial, sans-serif'),
            #TICKS SETUP OF COLORBAR
            nticks=10, ticks='outside',
            ticklen=1, tickwidth=1,
            showticklabels=True,
            tickangle=0, tickfont_size=12
        )
    ))

# Add images
figure.add_layout_image(
        dict(
            source=IMG_PATH,
            xref="x2 domain",
            yref="y2 domain",
            x=0,
            y=1,
            sizex=1,
            sizey=1,
            opacity=1,
            sizing = "stretch",
            layer="below")
)

#figure.update_layout(template="plotly_white")

g = go.FigureWidget(figure)


#LINK team to widget 
def update_fig(change):
    
    season = season_selector.value
    team = team_selector.value
    
    title = team + '\n' + "Season: " + str(season) + "  Relative to League Average for the Season "
    
    out1 = advanced_visualization(year=season,team_name=team,bin_width=1)
    f = gaussian_filter(out1, sigma=2)

    with g.batch_update():
            #g.
            g.data[0].z = f
            #g.data[1].x = x2
            g.layout.title = title
            #g.layout.xaxis.title = 'Delay in Minutes'
            #g.layout.yaxis.title = 'Number of Delays'
            
season_selector.observe(update_fig, 'value')
team_selector.observe(update_fig, 'value')

widgets.VBox([container,
              g])

File already Exists, loading from ../ift6758/data/PICKLE//2016_clean.pkl
num_teams 34
                team.name  x_bin  y_bin  shots_average  excess_shots
35597  New York Islanders   75.0   36.0       0.000745      0.000288
File already Exists, loading from ../ift6758/data/PICKLE//2017_clean.pkl
num_teams 35
                team.name  x_bin  y_bin  shots_average  excess_shots
36876  New York Islanders   75.0   36.0       0.000734     -0.001509
File already Exists, loading from ../ift6758/data/PICKLE//2017_clean.pkl
num_teams 35
                 team.name  x_bin  y_bin  shots_average  excess_shots
52729  Toronto Maple Leafs   75.0   36.0       0.000999     -0.001244
